In [63]:
import pandas as pd
from tools.hand_parser import *
from random import choice
import warnings 
warnings.filterwarnings('ignore')

In [46]:
%%time
df = pd.read_csv("hand_combinations.csv", index_col=0).astype('int8')
df.rename(index=lambda i: i.replace("'", ""), inplace=True)
print(f"Memory usage: {round(sum(df.memory_usage(deep=True)) / 1024 / 1024, 1)}MB\n")
all_hands = list(df.index.values)
df

Memory usage: 433.2MB

CPU times: total: 13.3 s
Wall time: 13.3 s


KeyboardInterrupt: 

In [ ]:
%%time
df['ranking'] = df.index
df['ranking'] = df['ranking'].apply(lambda x: parse(str(x)[1:-1].split(", ")))
df

In [ ]:
# SHOWING RANDOM hand with its ranking

# for _ in range(10):
#     hand = choice(all_hands)
#     print(hand, df.at[hand, 'ranking'])

In [ ]:
# df['ranking'].value_counts()

# CLASSIFICATION

##### Create rank_dict for renaming columns for future purposes in classification

In [ ]:
rankings = df['ranking'].value_counts().index.to_list()
# rankings.reverse()
# print(rankings)

rank_dict = {}
for i, el in enumerate(rankings):
    rank_dict[el] = i
rank_dict

In [ ]:
df['labels'] = df['ranking'].replace(rank_dict)
df

# Splitting DATASET

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
X = df.drop(['ranking', 'labels'], axis=1).to_numpy()
y = df['labels'].to_numpy()

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=1)

# Building RandomForestClassifier

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

In [ ]:
%%time
rfc_model = RandomForestClassifier(random_state=1, n_jobs=-1)
rfc_model.fit(X_train, y_train)
rfc_prediction = rfc_model.predict(X_test)
rfc_score = accuracy_score(y_test, rfc_prediction)
print(rfc_model)
print(rfc_score)

# Building Artificial Neural Networt

In [ ]:
import tensorflow as tf
from tensorflow.keras import Sequential, layers

In [ ]:
ANN = Sequential([
    layers.Flatten(input_shape=(52,)),
    
    layers.Dense(64, activation='relu'),
    
#     layers.Dropout(0.1),
    
    layers.Dense(len(rankings), activation='softmax')
])

ANN.compile(
    optimizer='adam',
    loss = 'sparse_categorical_crossentropy',
    metrics = ['accuracy']
)

ANN.fit(X_train, y_train, batch_size=256, epochs=5, validation_split=0.2)

In [ ]:
_, ann_acc = ANN.evaluate(X_test, y_test, batch_size=256)

In [ ]:
print(f"RFC(acc): {round(rfc_score * 100, 3)}%")
print(f"ANN(acc): {round(ann_acc * 100, 3)}%")
print(f"Impovement ANN over RFC: {round(ann_acc / rfc_score * 100 - 100, 2)}%")

In [ ]:
# dataset = tf.data.Dataset.from_tensor_slices((X, y))
# dataset = dataset.cache().batch(256).prefetch(tf.data.experimental.AUTOTUNE)
# _ = list(dataset.as_numpy_iterator())

# inputs = tf.keras.layers.Input((52,))
# x = tf.keras.layers.Dense(64, activation='relu')(inputs)
# x = tf.keras.layers.Dense(32, activation='relu')(x)
# outputs = tf.keras.layers.Dense(1, activation='softmax')(x)

# model = tf.keras.Model(inputs, outputs)
# model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
# model.fit(dataset, epochs=5)

#  <font color="gold">FINDING BEST MODEL ARCHITECTURE w/ KERAS TUNER</font>

In [50]:
from kerastuner.tuners import RandomSearch
import numpy as np

In [52]:
def build_model(hp):
    model = Sequential([
        layers.Flatten(input_shape=(52,)),

        layers.Dense(hp.Choice("Dense layer", list(range(100, 301, 10))), activation='relu'),

        #     layers.Dropout(0.1),

        layers.Dense(len(rankings), activation='softmax')
    ])
    
    model.compile(
        optimizer='adam',
        loss='sparse_categorical_crossentropy',
        metrics=['accuracy']
    )
    
    return model

tuner = RandomSearch(
    build_model,
    objective='val_loss',
    max_trials=10,
    overwrite=True
    )

tuner.search(X_train, y_train, validation_data=(X_test, y_test), epochs=5, batch_size=256)

Trial 10 Complete [00h 02m 35s]
val_loss: 0.0003154856385663152

Best val_loss So Far: 0.00012219283962622285
Total elapsed time: 00h 26m 22s
INFO:tensorflow:Oracle triggered exit


In [53]:
best_ann_model = tuner.get_best_models(1)[0]
best_hyp_par = tuner.get_best_hyperparameters(1)[0]
best_ann_model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 flatten (Flatten)           (None, 52)                0         
                                                                 
 dense (Dense)               (None, 300)               15900     
                                                                 
 dense_1 (Dense)             (None, 10)                3010      
                                                                 
Total params: 18,910
Trainable params: 18,910
Non-trainable params: 0
_________________________________________________________________


In [54]:
# best_ann_model.fit(X_train, y_train, epochs=5, batch_size=256, validation_split=0.1)
best_ann_model.evaluate(X_test, y_test, batch_size=256)

3046/3046 [==============================] - 8s 3ms/step - loss: 1.2219e-04 - accuracy: 1.0000


[0.00012219283962622285, 0.999984622001648]

In [68]:
list_of_models = tuner.get_best_models(10)
top_dense_layer_shapes = []
for model in list_of_models:
    top_dense_layer_shapes.append(model.get_layer('dense').output_shape[1])
#     print(model.get_layer('dense').output_shape[1])
top_dense_layer_shapes

In [71]:
dense_dict = {}
for i, m in enumerate(list_of_models):
    _, score = m.evaluate(X_test, y_test, batch_size=256)
    num_of_dense = top_dense_layer_shapes[i]
    dense_dict[str(num_of_dense)] = score

3046/3046 [==============================] - 7s 2ms/step - loss: 0.0073 - accuracy: 0.9984


In [93]:
dense_dict = {k: v for k, v in sorted(dense_dict.items(), key=lambda item: item[1], reverse=True)}
for k, v in dense_dict.items():
    print(f"Hidden Neurons(num) -> {k}: acc: {round(v, 12)}")

Hidden Neurons(num) -> 260: acc: 0.999990999699
Hidden Neurons(num) -> 270: acc: 0.999987185001
Hidden Neurons(num) -> 250: acc: 0.999985873699
Hidden Neurons(num) -> 300: acc: 0.999984622002
Hidden Neurons(num) -> 210: acc: 0.999982059002
Hidden Neurons(num) -> 190: acc: 0.999982059002
Hidden Neurons(num) -> 180: acc: 0.999973058701
Hidden Neurons(num) -> 160: acc: 0.999970495701
Hidden Neurons(num) -> 220: acc: 0.999957680702
Hidden Neurons(num) -> 110: acc: 0.998446822166
